In [ ]:
!pip install praw
from spacy.matcher import Matcher
import pandas as pd
import spacy
import praw
#pd.options.display.float_format = '{:.5f}'.format

In [ ]:
reddit = praw.Reddit(client_id='74UKCjQe_A-udw', client_secret='-1jNIfc3BM8eVr4b0zZXy74Xwag', user_agent='Project Spikes')
elections = reddit.subreddit('MtAugusta').search('title:[Election]', limit=150)

In [ ]:
#Build a DataFrame with those posts
posts = pd.DataFrame([[post.title, post.url, post.score, post.num_comments, post.created] for post in elections if post.title[:10].lower() == '[election]'],
                     columns=['title', 'url', 'score', 'num_comments', 'created'])
posts

In [ ]:
#define a get_comments function that returns all comments from a given URL
comments = pd.DataFrame()
def get_comments(url):
    submission = reddit.submission(url=url)
    for comment in submission.comments:
        try:
            comments.loc[comment.author.name, url] = '[' + comment.body + ']'
        except AttributeError:
            continue
    return comments
#For example:
for post in posts.index:
    get_comments(posts.url[post])
comments.to_csv('comments.csv')
comments

In [ ]:
nlp = spacy.load("en_core_web_sm")

#Build a matcher for numbers followed by punctuation
matcher = Matcher(nlp.vocab)
matcher.add('UserMat', None,
            [{"POS": "NUM"}, {"IS_PUNCT": True}, {"LIKE_NUM": False}])

In [ ]:
#IRRELEVANT: used for debugging
def tokenize(doc):
    for token in nlp(doc):
        print(token, token.pos_)

#Takes a username and counts every vote said user has casted
def count_votes(username):
    output = {}
    for comment in comments.loc[username].dropna():
        doc = nlp(str(comment).replace('\\n', ' ').replace('\\t', ' ').replace(')', '.'))
        matches = matcher(doc)
        
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]
            if span[:1].text == '1':
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 5
                except KeyError:
                    output[span[2:].text.lower()] = 5
            elif span[:1].text == '2':
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 3
                except KeyError:
                    output[span[2:].text.lower()] = 3
            else:
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 2
                except KeyError:
                    output[span[2:].text.lower()] = 2
    return output

test = count_votes('squareblob')
print(test)

In [ ]:
#Create a df with votes per candidate by voter
df = pd.DataFrame(index=comments.index)
for voter, _ in df.iterrows():
    ballots = count_votes(voter)
    for candidate in ballots.keys():
        df.loc[voter, candidate] = ballots[candidate]
df.to_csv('affinity.csv')

In [ ]:
#Create a df with voter participation per election (0 if not voted, 1 if voted)
participation = comments.T.notnull().astype('int')
for col in participation.columns:
    if participation[col].sum() < 4:
        participation.drop(col, axis=1, inplace=True)
participation.to_csv('participation.csv')

In [ ]:
count_votes('squareblob')

In [ ]:
#Same behaviour as count_votes but takes a URL instead
def count_votes_url(url):
    output = {}
    for comment in comments[url].dropna():
        doc = nlp(str(comment).replace('\\n', ' ').replace('\\t', ' ').replace(')', '.'))
        matches = matcher(doc)
        
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]
            if span[:1].text == '1':
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 5
                except KeyError:
                    output[span[2:].text.lower()] = 5
            elif span[:1].text == '2':
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 3
                except KeyError:
                    output[span[2:].text.lower()] = 3
            else:
                try:
                    output[span[2:].text.lower()] = output[span[2:].text.lower()] + 2
                except KeyError:
                    output[span[2:].text.lower()] = 2
    return output
count_votes_url('https://www.reddit.com/r/MtAugusta/comments/hd1437/election_judge_iii/')

In [ ]:
#Create a df2 with votes per candidate by post
df2 = pd.DataFrame(index=posts.url)
for url in df2.index:
    ballots = count_votes_url(url)
    for candidate in ballots.keys():
        df2.loc[url, candidate] = ballots[candidate]
df2.to_csv('popularity.csv')

In [ ]:
df2

In [ ]:
posts.set_index('url', inplace=True)

In [ ]:
for url in df2.index:
    df2.loc[url, 'time'] = posts.created[url]
df2['time'] = df2['time'].astype(int)
df2.sort_values(by='time', inplace=True)
df2['time'] = pd.to_datetime(df2['time'], unit='s')
df2.fillna(0, inplace=True)
df2

In [ ]:
testframe = pd.DataFrame({'A': [1,2,3,4,5,6], 'B':[0,1,0,1,0,1], 'C':[0,0,0,0,0,0]})

#Need a function to sum merge columns to avoid data fragmentation
def mergesum(df, parent, child):
    for row in range(df[parent].size):
        df[parent][row] = df[parent][row] + df[child][row]
    df.drop(child, axis=1, inplace=True)
    
def mergesums(df, parent, childs):
    for child in childs:
        try:
            mergesum(df, parent, child)
        except KeyError:
            continue
mergesums(testframe, 'A', ['B','C'])

In [ ]:
mergesums(df2, 'citylion', ['city', 'citypedeleon', 'citylion1'])
mergesums(df2, 'programmc', ['program', 'dchero', 'wrongaardvark', '\u2060programmc', "d'chero"])
mergesums(df2, 'ahrimanne', ['ahri', 'ahrimanazu', 'pamplamanazu', ''])
mergesums(df2, 'topher3001', ['topher', 'topha', 'toph'])
mergesums(df2, 'sandfalls', ['swissandham', 'swiss', 'swiss&ham'])
mergesums(df2, 'waffle4breakfast', ['waffle', 'waffles', 'pancake5lunch', 'crepe6dinner'])
mergesums(df2, 'rakkwal', ['rakk', 'rakkwall'])
mergesums(df2, 'kingcold64', ['kingcold', 'fladeedledoodle'])
mergesums(df2, 'henrydraton', ['henry', 'henrydration'])
mergesums(df2, 'godomasta', ['godo', '\u2060godomasta', 'godomasto', 'gododmasta'])
mergesum(df2, 'squareblob', 'square')

In [ ]:
df2cum = df2.iloc[:,:-1]
df2cum = df2cum.cumsum()
df2cum['time'] = df2['time'].dt.date
df2cum.set_index('time', inplace=True)
df2cum.to_csv('popularity.csv')
pd.set_option('max_columns', None)
df2cum

In [ ]:
#FINALLY! Start plotting the racing bars
!pip install bar_chart_race
import bar_chart_race as bcr
startdate = pd.to_datetime("2018-04-16").date()
enddate = pd.to_datetime("2019-12-08").date()
race = df2cum.loc[startdate:enddate]
bcr.bar_chart_race(race, n_bars=10, steps_per_period=60, period_length=800)